<a href="https://colab.research.google.com/github/Yongshilin/GIS/blob/main/GPXsToShapefile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Combine all GPX files to One Shapefile and Download

#### 1. Install packages

In [6]:
pip install geopandas shapely gpxpy

#### 2. Run this block of code to combine all the GPX files into a new shapefiles:

In [10]:
import os
import gpxpy
import geopandas as gpd
from shapely.geometry import LineString
import pandas as pd
import zipfile

def gpx_to_geodataframe(gpx_file):
    # Extract the trail name from the GPX file path
    trail_name = os.path.basename(gpx_file).replace('.gpx', '')

    with open(gpx_file, 'r') as file:
        gpx = gpxpy.parse(file)

    lines = []  # List to hold LineString geometries
    for track in gpx.tracks:
        for segment in track.segments:
            coords = [(point.longitude, point.latitude) for point in segment.points]
            if coords:  # Ensure there are coordinates to create a LineString
                line = LineString(coords)
                lines.append({
                    'geometry': line,
                    'trail_name': trail_name,  # Add trail name as an attribute
                    'elevation': [point.elevation for point in segment.points],  # Optional: Add elevation
                    'time': [point.time for point in segment.points]  # Optional: Add time
                })

    return gpd.GeoDataFrame(lines, crs="EPSG:4326")

def merge_gpx_files(gpx_files):
    all_lines = []
    for gpx_file in gpx_files:
        gdf = gpx_to_geodataframe(gpx_file)
        all_lines.append(gdf)

    return gpd.GeoDataFrame(pd.concat(all_lines, ignore_index=True))

def save_shapefile_as_zip(gdf, shapefile_name, output_folder):
    # Construct the shapefile path
    shapefile_path = os.path.join(output_folder, f"{shapefile_name}.shp")

    # Save as shapefile
    gdf.to_file(shapefile_path, driver="ESRI Shapefile")

    # Zip the shapefile components
    shapefile_zip = f"{shapefile_path}.zip"
    with zipfile.ZipFile(shapefile_zip, 'w') as zipf:
        for root, _, files in os.walk(output_folder):
            for file in files:
                if file.startswith(shapefile_name) and file.endswith(('.shp', '.shx', '.dbf', '.prj')):
                    zipf.write(os.path.join(root, file), arcname=file)

/usr/local/lib/python3.10/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Value '[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.95, 1.55, 1.16, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.18, 1.74, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0]' of field elevation has been truncated to 254 characters.  This warning will not be emitted any more for that layer.
  ogr_write(


#### 3. Modify your file names in this block of code:
a) First, create 2 new folders and name them:
- You will need to create a folder storing all the trails data in GPX format.

In this example: create a new folder named 'gpx_files' under folder 'content'. Then right click the 'gpx_files', select 'upload', and then choose all the GPX files you want to combine.
- You will need to create a new folder to save the combined GPX files in Shapefiles format.

In this example: create a folder named 'output_folder' under folder 'content'.

b) Second, modify your folder & file names:
- Fill in your folder path in code area 'gpx_directory'.

In this example:
Click the 'gpx_files', right click 'copy path', then paste it to replace the '/content/gpx_files' in this line of code: gpx_directory = '/content/gpx_files'(Don't drop the '' when you replace the path.)

- Fill in your folder path in code area 'output_folder'.

In this example:
Click the 'output_folder', right click 'copy path', then paste it to replace the '/content/output_folder' in this line of code: output_folder = "/content/output_folder" (Don't drop the '' when you replace the path.)

- Name your shapefile_name.

In this example:
We name our shapefile as "AIS_trails", so the code is: shapefile_name = "AIS_trails". You can rename it.

- After running the code till here, you can find the Shapefile Zip(AIS_trails.shp.zip) is saved in the 'output_folder'.


In [ ]:
# Automatically collect all GPX files from a directory
gpx_directory = '/content/gpx_files'  # Replace with your directory containing GPX files
gpx_files = [os.path.join(gpx_directory, f) for f in os.listdir(gpx_directory) if f.endswith('.gpx')]

output_folder = "/content/output_folder"  # Specify your output directory
shapefile_name = "AIS_trails"  # Specify your shapefile name

# Merge GPX files and save as a shapefile zip
merged_gdf = merge_gpx_files(gpx_files)
save_shapefile_as_zip(merged_gdf, shapefile_name, output_folder)
# the '.shp.zip' saved in the output_folder is the shapefile combining all our trails
# you can manually download it to your device, or you can run the code below to do it.

#### Download the shpefile from the 'output_folder' to local computer:
- Right click the 'AIS_trails.ship.zip', click 'copy path', then put the path in the below code files.download()

In this example: add our file path (/content/output_folder/AIS_trails.shp.zip) inside the brackets, remeber to add the ' '.

In [12]:
# Download shpefile zip file to the computer
from google.colab import files
files.download('/content/output_folder/AIS_trails.shp.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

_____
#### If you want to make sure it shows the trails in line format instead of points in the map, you can run the code below for checking:

In [11]:
#Verify Geometry Types
#If the output shows Point instead of LineString, then the problem lies in how the geometries are being created or merged.
print(merged_gdf.geometry.geom_type)

0     LineString
1     LineString
2     LineString
3     LineString
4     LineString
         ...    
79    LineString
80    LineString
81    LineString
82    LineString
83    LineString
Length: 84, dtype: object


In [ ]:
#Check for Valid Geometries
print(merged_gdf.geometry.is_valid)

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
dtype: bool


In [ ]:
## DO NOT RUN THIS CODE IF THE shapefiles WORKS!
#If any of the geometries are invalid, you can try to fix them using the buffer(0) method, which can help to correct minor issues:
merged_gdf['geometry'] = merged_gdf.geometry.buffer(0)